In [ ]:
!pip install sentence_transformers
!pip install rouge-score!
import sys
import os
from typing import List, Dict, Tuple, Any, Optional, Union, Callable
import json
import logging
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextGenerationPipeline
from transformers import LogitsProcessorList, StoppingCriteriaList, StoppingCriteria
from transformers.generation.logits_process import (
    LogitsProcessor,
    RepetitionPenaltyLogitsProcessor,
    TopKLogitsWarper,
    TopPLogitsWarper,
)

import pandas as pd
import re
import logging
from typing import Dict, List, Tuple, Set, Optional, Union

import spacy
from spacy.tokens import Doc, Span, Token

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
PII_PATTERNS = {
    'email': r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
    'phone_us': r'\b(\+\d{1,2}\s?)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b',
    'ssn': r'\b\d{3}-\d{2}-\d{4}\b',
    'credit_card': r'\b(?:\d{4}[-\s]?){3}\d{4}\b',
    'ip_address': r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b',
    'customer_id': r'\bCUST-\d{4}-\d{4}\b',
    'url': r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+[/\w\.-]*\??[-\w&=%]*',
    'date_of_birth': r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b',
    'zip_code': r'\b\d{5}(?:-\d{4})?\b',
}

# Compiled regex patterns for faster matching
COMPILED_PATTERNS = {name: re.compile(pattern) for name, pattern in PII_PATTERNS.items()}

# Entity types to detect with NER
NER_ENTITY_TYPES = [
    'PERSON',
    'ORG',
    'GPE',  # Geo-political entities (countries, cities)
    'LOC',  # Non-GPE locations
    'FACILITY',
    'PRODUCT',
    'EVENT',
    'WORK_OF_ART',
    'LAW',
    'LANGUAGE',
    'FAC',  # Buildings, airports
    'NORP'  # Nationalities, religious groups
]

# Placeholder formats
def get_placeholder(entity_type, index=None):
    """Generate a placeholder for the detected entity type."""
    if index is not None:
        return f"[{entity_type}_{index}]"
    return f"[{entity_type}]"

In [ ]:
def load_configuration(config_path: str) -> Dict[str, Any]:
    """Load pipeline configuration from a JSON file."""
    with open(config_path, 'r') as f:
        config = json.load(f)
    return config

def save_results(results, output_path: str):
    """Save results to a file."""
    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    with open(output_path, 'w') as f:
        if isinstance(results, dict):
            json.dump(results, f, indent=2)
        else:
            f.write(results)

    logger.info(f"Results saved to {output_path}")

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import spacy

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class PrivacyEvaluator:
    """
    Evaluates privacy metrics for synthetic text generation.
    """

    def __init__(
        self,
        embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2",
        similarity_thresholds: List[float] = None,
        device: str = None,
        batch_size: int = 32,
        use_spacy: bool = True,
        spacy_model: str = "en_core_web_sm"
    ):
        """
        Initialize the privacy evaluator.

        Args:
            embedding_model: SentenceTransformer model to use for embeddings.
            similarity_thresholds: List of thresholds for similarity analysis.
            device: Device to use for embeddings (cuda, cpu, mps).
            batch_size: Batch size for computing embeddings.
            use_spacy: Whether to use spaCy for entity extraction.
            spacy_model: spaCy model to use for entity extraction.
        """
        self.similarity_thresholds = similarity_thresholds or [0.8, 0.85, 0.9, 0.95, 0.98]
        self.batch_size = batch_size

        # Load embedding model
        try:
            logger.info(f"Loading embedding model: {embedding_model}")
            self.embedding_model = SentenceTransformer(embedding_model, device=device)
        except Exception as e:
            logger.error(f"Failed to load embedding model: {str(e)}")
            raise

        # Load spaCy model
        self.use_spacy = use_spacy
        if use_spacy:
            try:
                self.nlp = spacy.load(spacy_model)
                logger.info(f"Loaded spaCy model: {spacy_model}")
            except Exception as e:
                logger.error(f"Failed to load spaCy model: {str(e)}")
                logger.warning("Downloading spaCy model...")
                spacy.cli.download(spacy_model)
                self.nlp = spacy.load(spacy_model)

    def evaluate_pii_leakage(
        self,
        original_data: pd.DataFrame,
        generated_texts: List[str],
        text_column: str,
        pii_metadata_column: str = "pii_metadata",
        ignore_case: bool = True
    ) -> Dict[str, Any]:
        """
        Evaluate PII leakage rates in generated texts.

        Args:
            original_data: DataFrame with original data and PII metadata.
            generated_texts: List of generated texts to evaluate.
            text_column: Column containing the text in original_data.
            pii_metadata_column: Column containing PII metadata in original_data.
            ignore_case: Whether to ignore case when matching PII.

        Returns:
            Dictionary with PII leakage metrics.
        """
        if len(original_data) != len(generated_texts):
            raise ValueError("Number of original samples and generated texts must match")

        # Track PII leakage
        total_samples = len(original_data)
        total_with_pii = 0
        total_leaked = 0
        pii_types_leaked = {}
        leaked_samples = []

        # Evaluate each sample
        for i, (_, row) in enumerate(original_data.iterrows()):
            # Skip samples without PII
            pii_metadata = row.get(pii_metadata_column, {})
            if not pii_metadata or not isinstance(pii_metadata, dict) or not pii_metadata.get("injected", False):
                continue

            total_with_pii += 1
            generated_text = generated_texts[i]

            # Check for PII leakage
            sample_leaked = False
            leaked_pii_types = []
            leaked_values = []

            for injection in pii_metadata.get("injections", []):
                pii_type = injection.get("type", "unknown")
                pii_value = injection.get("value", "")

                # Skip empty values
                if not pii_value:
                    continue

                # Check if the PII value is in the generated text
                if ignore_case:
                    found = pii_value.lower() in generated_text.lower()
                else:
                    found = pii_value in generated_text

                if found:
                    sample_leaked = True
                    leaked_pii_types.append(pii_type)
                    leaked_values.append(pii_value)

                    # Update type-specific counts
                    pii_types_leaked[pii_type] = pii_types_leaked.get(pii_type, 0) + 1

            # Record if this sample leaked PII
            if sample_leaked:
                total_leaked += 1
                leaked_samples.append({
                    "sample_idx": i,
                    "pii_types": leaked_pii_types,
                    "leaked_values": leaked_values
                })

        # Calculate metrics
        if total_with_pii > 0:
            leakage_rate = total_leaked / total_with_pii
            per_type_rate = {pii_type: count / total_with_pii for pii_type, count in pii_types_leaked.items()}
        else:
            leakage_rate = 0.0
            per_type_rate = {}

        return {
            "total_samples": total_samples,
            "samples_with_pii": total_with_pii,
            "samples_leaked_pii": total_leaked,
            "pii_leakage_rate": leakage_rate,
            "per_type_leakage_rate": per_type_rate,
            "leaked_samples": leaked_samples
        }

    def evaluate_canary_leakage(
        self,
        original_data: pd.DataFrame,
        generated_texts: List[str],
        text_column: str,
        canary_metadata_column: str = "canary_metadata",
        similarity_threshold: float = 0.95,
        exact_match: bool = True,
        ignore_case: bool = True
    ) -> Dict[str, Any]:
        """
        Evaluate canary phrase leakage rates in generated texts.

        Args:
            original_data: DataFrame with original data and canary metadata.
            generated_texts: List of generated texts to evaluate.
            text_column: Column containing the text in original_data.
            canary_metadata_column: Column containing canary metadata in original_data.
            similarity_threshold: Threshold for similarity-based leakage detection.
            exact_match: Whether to require exact matches (vs. similarity).
            ignore_case: Whether to ignore case when matching canaries.

        Returns:
            Dictionary with canary leakage metrics.
        """
        if len(original_data) != len(generated_texts):
            raise ValueError("Number of original samples and generated texts must match")

        # Track canary leakage
        total_samples = len(original_data)
        total_with_canary = 0
        total_leaked_exact = 0
        total_leaked_similar = 0
        canary_phrase_counts = {}
        leaked_samples = []

        # Get all unique canary phrases for batch embedding
        all_canaries = set()
        for _, row in original_data.iterrows():
            canary_metadata = row.get(canary_metadata_column, {})
            if canary_metadata and isinstance(canary_metadata, dict) and canary_metadata.get("injected", False):
                canary = canary_metadata.get("canary", "")
                if canary:
                    all_canaries.add(canary)

        # Compute embeddings for all canary phrases
        if all_canaries and not exact_match:
            canary_phrases = list(all_canaries)
            canary_embeddings = self.embedding_model.encode(
                canary_phrases,
                batch_size=self.batch_size,
                show_progress_bar=len(canary_phrases) > 10
            )
            canary_embed_map = {phrase: embedding for phrase, embedding in zip(canary_phrases, canary_embeddings)}
        else:
            canary_embed_map = {}

        # Evaluate each sample
        for i, (_, row) in enumerate(original_data.iterrows()):
            # Skip samples without canary phrases
            canary_metadata = row.get(canary_metadata_column, {})
            if not canary_metadata or not isinstance(canary_metadata, dict) or not canary_metadata.get("injected", False):
                continue

            total_with_canary += 1
            generated_text = generated_texts[i]
            canary_phrase = canary_metadata.get("canary", "")

            # Skip empty canary phrases
            if not canary_phrase:
                continue

            # Update canary phrase counts
            canary_phrase_counts[canary_phrase] = canary_phrase_counts.get(canary_phrase, 0) + 1

            # Check for exact match
            leaked_exact = False
            if ignore_case:
                if canary_phrase.lower() in generated_text.lower():
                    leaked_exact = True
            else:
                if canary_phrase in generated_text:
                    leaked_exact = True

            # Check for similarity-based leakage if exact match wasn't found
            leaked_similar = False
            similarity_score = 0.0

            if not leaked_exact and not exact_match:
                # Get embedding for generated text
                gen_embedding = self.embedding_model.encode(generated_text, batch_size=1)

                # Get embedding for this canary phrase
                canary_embedding = canary_embed_map.get(canary_phrase)

                if canary_embedding is not None:
                    # Calculate similarity
                    similarity_score = cosine_similarity(
                        [gen_embedding],
                        [canary_embedding]
                    )[0][0]

                    if similarity_score >= similarity_threshold:
                        leaked_similar = True

            # Record leakage
            if leaked_exact:
                total_leaked_exact += 1
                leaked_samples.append({
                    "sample_idx": i,
                    "canary_phrase": canary_phrase,
                    "match_type": "exact",
                    "similarity": 1.0
                })
            elif leaked_similar:
                total_leaked_similar += 1
                leaked_samples.append({
                    "sample_idx": i,
                    "canary_phrase": canary_phrase,
                    "match_type": "similar",
                    "similarity": float(similarity_score)
                })

        # Calculate metrics
        if total_with_canary > 0:
            exact_leakage_rate = total_leaked_exact / total_with_canary
            similar_leakage_rate = total_leaked_similar / total_with_canary
            total_leakage_rate = (total_leaked_exact + total_leaked_similar) / total_with_canary
        else:
            exact_leakage_rate = 0.0
            similar_leakage_rate = 0.0
            total_leakage_rate = 0.0

        return {
            "total_samples": total_samples,
            "samples_with_canary": total_with_canary,
            "samples_leaked_exact": total_leaked_exact,
            "samples_leaked_similar": total_leaked_similar,
            "exact_leakage_rate": exact_leakage_rate,
            "similar_leakage_rate": similar_leakage_rate,
            "total_leakage_rate": total_leakage_rate,
            "canary_phrase_counts": canary_phrase_counts,
            "leaked_samples": leaked_samples
        }

    def evaluate_similarity_distribution(
        self,
        original_data: pd.DataFrame,
        generated_texts: List[str],
        text_column: str,
        analyze_chunks: bool = True,
        chunk_size: int = 100
    ) -> Dict[str, Any]:
        """
        Evaluate similarity between original and generated texts.

        Args:
            original_data: DataFrame with original data.
            generated_texts: List of generated texts to evaluate.
            text_column: Column containing the text in original_data.
            analyze_chunks: Whether to analyze text chunks for more granular similarity.
            chunk_size: Size of text chunks for chunk-based analysis.

        Returns:
            Dictionary with similarity metrics.
        """
        if len(original_data) != len(generated_texts):
            raise ValueError("Number of original samples and generated texts must match")

        # Extract original texts
        original_texts = original_data['body'].tolist()

        # Compute embeddings for whole texts
        logger.info("Computing embeddings for original texts...")
        original_embeddings = self.embedding_model.encode(
            original_texts,
            batch_size=self.batch_size,
            show_progress_bar=len(original_texts) > 10
        )

        logger.info("Computing embeddings for generated texts...")
        generated_embeddings = self.embedding_model.encode(
            generated_texts,
            batch_size=self.batch_size,
            show_progress_bar=len(generated_texts) > 10
        )

        # Calculate similarities
        similarities = []
        for i in range(len(original_embeddings)):
            similarity = cosine_similarity(
                [original_embeddings[i]],
                [generated_embeddings[i]]
            )[0][0]
            similarities.append(float(similarity))

        # Calculate metrics for whole texts
        whole_text_metrics = {
            "mean_similarity": float(np.mean(similarities)),
            "median_similarity": float(np.median(similarities)),
            "min_similarity": float(np.min(similarities)),
            "max_similarity": float(np.max(similarities)),
            "std_deviation": float(np.std(similarities))
        }

        # Calculate thresholded counts
        threshold_counts = {}
        for threshold in self.similarity_thresholds:
            count = sum(1 for sim in similarities if sim >= threshold)
            rate = count / len(similarities) if similarities else 0
            threshold_counts[f"above_{threshold:.2f}"] = {
                "count": count,
                "rate": rate
            }

        # Analyze chunks if requested
        chunk_metrics = {}
        if analyze_chunks:
            chunk_similarities = []
            high_similarity_chunks = []

            for i, (orig_text, gen_text) in enumerate(zip(original_texts, generated_texts)):
                # Skip if either text is too short
                if len(orig_text) < chunk_size or len(gen_text) < chunk_size:
                    continue

                # Create chunks
                orig_chunks = [orig_text[j:j+chunk_size] for j in range(0, len(orig_text), chunk_size)]
                gen_chunks = [gen_text[j:j+chunk_size] for j in range(0, len(gen_text), chunk_size)]

                # Calculate similarities between chunks
                for orig_idx, orig_chunk in enumerate(orig_chunks):
                    orig_embed = self.embedding_model.encode(orig_chunk, batch_size=1)

                    for gen_idx, gen_chunk in enumerate(gen_chunks):
                        gen_embed = self.embedding_model.encode(gen_chunk, batch_size=1)

                        chunk_sim = cosine_similarity([orig_embed], [gen_embed])[0][0]
                        chunk_similarities.append(float(chunk_sim))

                        # Record high similarity chunks
                        if chunk_sim >= 0.95:
                            high_similarity_chunks.append({
                                "sample_idx": i,
                                "orig_chunk_idx": orig_idx,
                                "gen_chunk_idx": gen_idx,
                                "orig_chunk": orig_chunk[:100] + "..." if len(orig_chunk) > 100 else orig_chunk,
                                "gen_chunk": gen_chunk[:100] + "..." if len(gen_chunk) > 100 else gen_chunk,
                                "similarity": float(chunk_sim)
                            })

            # Calculate chunk-level metrics
            if chunk_similarities:
                chunk_metrics = {
                    "mean_chunk_similarity": float(np.mean(chunk_similarities)),
                    "median_chunk_similarity": float(np.median(chunk_similarities)),
                    "min_chunk_similarity": float(np.min(chunk_similarities)),
                    "max_chunk_similarity": float(np.max(chunk_similarities)),
                    "std_deviation": float(np.std(chunk_similarities)),
                    "high_similarity_chunks": high_similarity_chunks[:20]  # Limit to first 20
                }

                # Calculate thresholded counts for chunks
                chunk_threshold_counts = {}
                for threshold in self.similarity_thresholds:
                    count = sum(1 for sim in chunk_similarities if sim >= threshold)
                    rate = count / len(chunk_similarities)
                    chunk_threshold_counts[f"above_{threshold:.2f}"] = {
                        "count": count,
                        "rate": rate
                    }

                chunk_metrics["threshold_counts"] = chunk_threshold_counts

        return {
            "whole_text_metrics": whole_text_metrics,
            "threshold_counts": threshold_counts,
            "chunk_metrics": chunk_metrics,
            "all_similarities": similarities
        }

    def run_adversarial_memorization_test(
        self,
        canary_phrases: List[str],
        adversarial_prompts: List[str],
        generated_texts: List[str],
        exact_match: bool = True,
        ignore_case: bool = True,
        similarity_threshold: float = 0.98
    ) -> Dict[str, Any]:
        """
        Evaluate resistance to adversarial prompts designed to elicit memorized content.

        Args:
            canary_phrases: List of canary phrases to test for.
            adversarial_prompts: List of prompts designed to elicit the canaries.
            generated_texts: List of texts generated from the adversarial prompts.
            exact_match: Whether to require exact matches (vs. similarity).
            ignore_case: Whether to ignore case when matching canaries.
            similarity_threshold: Threshold for similarity-based leakage detection.

        Returns:
            Dictionary with adversarial test metrics.
        """
        if len(adversarial_prompts) != len(generated_texts):
            raise ValueError("Number of adversarial prompts and generated texts must match")

        # Compute embeddings for canary phrases
        canary_embeddings = None
        if not exact_match:
            canary_embeddings = self.embedding_model.encode(
                canary_phrases,
                batch_size=self.batch_size,
                show_progress_bar=len(canary_phrases) > 10
            )

        # Track leakage
        total_prompts = len(adversarial_prompts)
        leaked_canaries = []

        for i, (prompt, generated_text) in enumerate(zip(adversarial_prompts, generated_texts)):
            # Check each canary phrase
            for j, canary in enumerate(canary_phrases):
                # Check for exact match
                leaked_exact = False
                if ignore_case:
                    if canary.lower() in generated_text.lower():
                        leaked_exact = True
                else:
                    if canary in generated_text:
                        leaked_exact = True

                # Check for similarity-based leakage if exact match wasn't found
                leaked_similar = False
                similarity_score = 0.0

                if not leaked_exact and not exact_match:
                    # Get embedding for generated text
                    gen_embedding = self.embedding_model.encode(generated_text, batch_size=1)

                    # Calculate similarity
                    if canary_embeddings is not None:
                        similarity_score = cosine_similarity(
                            [gen_embedding],
                            [canary_embeddings[j]]
                        )[0][0]

                        if similarity_score >= similarity_threshold:
                            leaked_similar = True

                # Record leakage
                if leaked_exact or leaked_similar:
                    leaked_canaries.append({
                        "prompt_idx": i,
                        "canary_idx": j,
                        "prompt": prompt[:100] + "..." if len(prompt) > 100 else prompt,
                        "canary": canary[:100] + "..." if len(canary) > 100 else canary,
                        "match_type": "exact" if leaked_exact else "similar",
                        "similarity": 1.0 if leaked_exact else float(similarity_score)
                    })

        # Calculate metrics
        success_rate = len(leaked_canaries) / (total_prompts * len(canary_phrases))

        return {
            "total_tests": total_prompts * len(canary_phrases),
            "successful_elicitations": len(leaked_canaries),
            "success_rate": success_rate,
            "leaked_canaries": leaked_canaries
        }

In [ ]:
import math
from collections import Counter
from typing import List, Dict, Tuple, Set, Any, Union, Optional

import numpy as np
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import jensenshannon
import pandas as pd

try:
    from rouge_score import rouge_scorer
    ROUGE_AVAILABLE = True
except ImportError:
    ROUGE_AVAILABLE = False

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class UtilityEvaluator:
    """
    Evaluates utility metrics for synthetic text generation.
    """

    def __init__(
        self,
        embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2",
        use_perplexity: bool = True,
        gpt2_model: str = "gpt2",
        device: str = None,
        batch_size: int = 8
    ):
        """
        Initialize the utility evaluator.

        Args:
            embedding_model: SentenceTransformer model to use for embeddings.
            use_perplexity: Whether to calculate perplexity metrics.
            gpt2_model: GPT-2 model to use for perplexity calculation.
            device: Device to use for models (cuda, cpu, mps).
            batch_size: Batch size for computing embeddings and perplexity.
        """
        self.batch_size = batch_size

        # Determine device
        if device is None:
            if torch.cuda.is_available():
                self.device = "cuda"
            elif hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
                self.device = "mps"
            else:
                self.device = "cpu"
        else:
            self.device = device

        logger.info(f"Using device: {self.device}")

        # Load embedding model
        try:
            logger.info(f"Loading embedding model: {embedding_model}")
            self.embedding_model = SentenceTransformer(embedding_model, device=self.device)
        except Exception as e:
            logger.error(f"Failed to load embedding model: {str(e)}")
            raise

        # Load GPT-2 model if perplexity is requested
        self.use_perplexity = use_perplexity
        if use_perplexity:
            try:
                logger.info(f"Loading GPT-2 model: {gpt2_model}")
                self.gpt2_tokenizer = GPT2TokenizerFast.from_pretrained(gpt2_model)
                # Set pad token to eos token
                self.gpt2_tokenizer.pad_token = self.gpt2_tokenizer.eos_token
                self.gpt2_model = GPT2LMHeadModel.from_pretrained(gpt2_model).to(self.device)
                self.gpt2_model.eval()
            except Exception as e:
                logger.error(f"Failed to load GPT-2 model: {str(e)}")
                self.use_perplexity = False
                logger.warning("Perplexity calculations will be skipped")

    def calculate_perplexity(self, texts: List[str]) -> Dict[str, Any]:
        """
        Calculate perplexity for a list of texts using GPT-2.

        Args:
            texts: List of texts to calculate perplexity for.

        Returns:
            Dictionary with perplexity metrics.
        """
        if not self.use_perplexity:
            return {"error": "Perplexity calculation is disabled"}

        # Calculate perplexity
        perplexities = []

        for i in range(0, len(texts), self.batch_size):
            batch_texts = texts[i:i+self.batch_size]

            encodings = self.gpt2_tokenizer(
                batch_texts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=1024
            ).to(self.device)

            with torch.no_grad():
                # We need to provide labels for loss calculation
                input_ids = encodings.input_ids
                outputs = self.gpt2_model(input_ids=input_ids, labels=input_ids)
                if outputs.loss is not None:
                    log_likelihood = outputs.loss.item() * input_ids.size(1)
                else:
                    # Skip this batch if loss is None
                    continue

                # Calculate perplexity for each text in the batch
                for j, text in enumerate(batch_texts):
                    # Tokenize individual text
                    input_ids = self.gpt2_tokenizer.encode(text, return_tensors="pt").to(self.device)

                    # Skip empty texts
                    if input_ids.size(1) <= 1:
                        continue

                    # Calculate perplexity
                    with torch.no_grad():
                        outputs = self.gpt2_model(input_ids, labels=input_ids)
                        neg_log_likelihood = outputs.loss.item()
                        ppl = math.exp(neg_log_likelihood)
                        perplexities.append(ppl)

        # Calculate metrics
        if perplexities:
            result = {
                "mean_perplexity": float(np.mean(perplexities)),
                "median_perplexity": float(np.median(perplexities)),
                "min_perplexity": float(np.min(perplexities)),
                "max_perplexity": float(np.max(perplexities)),
                "std_deviation": float(np.std(perplexities))
            }
        else:
            result = {"error": "Failed to calculate perplexity for any text"}

        return result

    def calculate_semantic_similarity(
        self,
        original_texts: List[str],
        generated_texts: List[str]
    ) -> Dict[str, Any]:
        """
        Calculate semantic similarity between original and generated texts.

        Args:
            original_texts: List of original texts.
            generated_texts: List of generated texts.

        Returns:
            Dictionary with semantic similarity metrics.
        """
        if len(original_texts) != len(generated_texts):
            return {"error": "Number of original and generated texts must match"}

        # Calculate embeddings
        logger.info("Computing embeddings for original texts...")
        original_embeddings = self.embedding_model.encode(
            original_texts,
            batch_size=self.batch_size,
            show_progress_bar=len(original_texts) > 10
        )

        logger.info("Computing embeddings for generated texts...")
        generated_embeddings = self.embedding_model.encode(
            generated_texts,
            batch_size=self.batch_size,
            show_progress_bar=len(generated_texts) > 10
        )

        # Calculate similarities
        similarities = []
        for i in range(len(original_embeddings)):
            similarity = cosine_similarity(
                [original_embeddings[i]],
                [generated_embeddings[i]]
            )[0][0]
            similarities.append(float(similarity))

        # Calculate metrics
        result = {
            "mean_similarity": float(np.mean(similarities)),
            "median_similarity": float(np.median(similarities)),
            "min_similarity": float(np.min(similarities)),
            "max_similarity": float(np.max(similarities)),
            "std_deviation": float(np.std(similarities))
        }

        return result

    def calculate_rouge_scores(
        self,
        original_texts: List[str],
        generated_texts: List[str]
    ) -> Dict[str, Any]:
        """
        Calculate ROUGE scores between original and generated texts.

        Args:
            original_texts: List of original texts.
            generated_texts: List of generated texts.

        Returns:
            Dictionary with ROUGE score metrics.
        """
        if not ROUGE_AVAILABLE:
            return {"error": "rouge_score library not available"}

        if len(original_texts) != len(generated_texts):
            return {"error": "Number of original and generated texts must match"}

        # Initialize ROUGE scorer
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

        # Calculate scores
        scores = []
        for original, generated in zip(original_texts, generated_texts):
            score = scorer.score(original, generated)
            scores.append({
                'rouge1_precision': score['rouge1'].precision,
                'rouge1_recall': score['rouge1'].recall,
                'rouge1_fmeasure': score['rouge1'].fmeasure,
                'rouge2_precision': score['rouge2'].precision,
                'rouge2_recall': score['rouge2'].recall,
                'rouge2_fmeasure': score['rouge2'].fmeasure,
                'rougeL_precision': score['rougeL'].precision,
                'rougeL_recall': score['rougeL'].recall,
                'rougeL_fmeasure': score['rougeL'].fmeasure
            })

        # Calculate averages
        avg_scores = {}
        for metric in scores[0].keys():
            avg_scores[metric] = float(np.mean([s[metric] for s in scores]))

        return {
            "average_scores": avg_scores,
            "all_scores": scores[:10]  # Return first 10 individual scores
        }

    def calculate_n_gram_diversity(
        self,
        original_texts: List[str],
        generated_texts: List[str],
        n_values: List[int] = None
    ) -> Dict[str, Any]:
        """
        Calculate n-gram diversity statistics.

        Args:
            original_texts: List of original texts.
            generated_texts: List of generated texts.
            n_values: List of n-gram sizes to calculate.

        Returns:
            Dictionary with n-gram diversity metrics.
        """
        n_values = n_values or [1, 2, 3]

        def get_n_grams(texts: List[str], n: int) -> Counter:
            """Get n-gram counts for a list of texts."""
            n_grams = Counter()
            for text in texts:
                tokens = text.lower().split()
                if len(tokens) < n:
                    continue
                text_n_grams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
                n_grams.update(text_n_grams)
            return n_grams

        # Calculate n-gram distributions
        result = {}
        for n in n_values:
            original_n_grams = get_n_grams(original_texts, n)
            generated_n_grams = get_n_grams(generated_texts, n)

            # Calculate unique n-grams
            unique_original = len(original_n_grams)
            unique_generated = len(generated_n_grams)
            total_original = sum(original_n_grams.values())
            total_generated = sum(generated_n_grams.values())

            # Calculate probabilities for Jensen-Shannon divergence
            if total_original > 0 and total_generated > 0 and unique_original > 0 and unique_generated > 0:
                # Get sets of n-grams
                all_n_grams = set(original_n_grams.keys()) | set(generated_n_grams.keys())

                # Calculate probability distributions
                orig_probs = np.array([original_n_grams.get(n_gram, 0) / total_original for n_gram in all_n_grams])
                gen_probs = np.array([generated_n_grams.get(n_gram, 0) / total_generated for n_gram in all_n_grams])

                # Add smoothing for zero probabilities
                orig_probs = np.maximum(orig_probs, 1e-10)
                gen_probs = np.maximum(gen_probs, 1e-10)

                # Normalize
                orig_probs = orig_probs / np.sum(orig_probs)
                gen_probs = gen_probs / np.sum(gen_probs)

                # Calculate Jensen-Shannon divergence
                jsd = jensenshannon(orig_probs, gen_probs)

                # Calculate normalized entropy
                generated_entropy = -np.sum(gen_probs * np.log2(gen_probs)) / np.log2(len(gen_probs))
                original_entropy = -np.sum(orig_probs * np.log2(orig_probs)) / np.log2(len(orig_probs))

                result[f"{n}-gram"] = {
                    "unique_original": unique_original,
                    "unique_generated": unique_generated,
                    "total_original": total_original,
                    "total_generated": total_generated,
                    "unique_ratio_original": unique_original / total_original if total_original > 0 else 0,
                    "unique_ratio_generated": unique_generated / total_generated if total_generated > 0 else 0,
                    "jensen_shannon_divergence": float(jsd),
                    "original_entropy_normalized": float(original_entropy),
                    "generated_entropy_normalized": float(generated_entropy)
                }
            else:
                result[f"{n}-gram"] = {
                    "error": "Insufficient data for n-gram diversity calculation"
                }

        return result

    def calculate_output_statistics(
        self,
        original_texts: List[str],
        generated_texts: List[str]
    ) -> Dict[str, Any]:
        """
        Calculate basic output statistics.

        Args:
            original_texts: List of original texts.
            generated_texts: List of generated texts.

        Returns:
            Dictionary with output statistics.
        """
        # Calculate lengths
        original_lengths = [len(text.split()) for text in original_texts]
        generated_lengths = [len(text.split()) for text in generated_texts]

        # Calculate character lengths
        original_char_lengths = [len(text) for text in original_texts]
        generated_char_lengths = [len(text) for text in generated_texts]

        # Calculate lengths of sentences
        sentence_pattern = r'[.!?]+\s+'
        original_sentence_counts = [len(re.split(sentence_pattern, text)) for text in original_texts]
        generated_sentence_counts = [len(re.split(sentence_pattern, text)) for text in generated_texts]

        # Count placeholder tokens
        placeholder_pattern = r'\[(REDACTED|PII|NAME|EMAIL|PHONE|ADDRESS|SSN|CREDIT_CARD|PERSON|ORG|GPE|LOC|\w+_\d+)\]'
        placeholder_counts = [len(re.findall(placeholder_pattern, text)) for text in generated_texts]

        result = {
            "word_count": {
                "original_mean": float(np.mean(original_lengths)),
                "original_median": float(np.median(original_lengths)),
                "original_min": float(np.min(original_lengths)),
                "original_max": float(np.max(original_lengths)),
                "original_std": float(np.std(original_lengths)),
                "generated_mean": float(np.mean(generated_lengths)),
                "generated_median": float(np.median(generated_lengths)),
                "generated_min": float(np.min(generated_lengths)),
                "generated_max": float(np.max(generated_lengths)),
                "generated_std": float(np.std(generated_lengths)),
                "ratio_mean": float(np.mean(generated_lengths) / np.mean(original_lengths)) if np.mean(original_lengths) > 0 else 0
            },
            "char_count": {
                "original_mean": float(np.mean(original_char_lengths)),
                "original_median": float(np.median(original_char_lengths)),
                "original_min": float(np.min(original_char_lengths)),
                "original_max": float(np.max(original_char_lengths)),
                "original_std": float(np.std(original_char_lengths)),
                "generated_mean": float(np.mean(generated_char_lengths)),
                "generated_median": float(np.median(generated_char_lengths)),
                "generated_min": float(np.min(generated_char_lengths)),
                "generated_max": float(np.max(generated_char_lengths)),
                "generated_std": float(np.std(generated_char_lengths)),
                "ratio_mean": float(np.mean(generated_char_lengths) / np.mean(original_char_lengths)) if np.mean(original_char_lengths) > 0 else 0
            },
            "sentence_count": {
                "original_mean": float(np.mean(original_sentence_counts)),
                "original_median": float(np.median(original_sentence_counts)),
                "original_min": float(np.min(original_sentence_counts)),
                "original_max": float(np.max(original_sentence_counts)),
                "original_std": float(np.std(original_sentence_counts)),
                "generated_mean": float(np.mean(generated_sentence_counts)),
                "generated_median": float(np.median(generated_sentence_counts)),
                "generated_min": float(np.min(generated_sentence_counts)),
                "generated_max": float(np.max(generated_sentence_counts)),
                "generated_std": float(np.std(generated_sentence_counts)),
                "ratio_mean": float(np.mean(generated_sentence_counts) / np.mean(original_sentence_counts)) if np.mean(original_sentence_counts) > 0 else 0
            },
            "placeholder_count": {
                "mean": float(np.mean(placeholder_counts)),
                "median": float(np.median(placeholder_counts)),
                "min": float(np.min(placeholder_counts)),
                "max": float(np.max(placeholder_counts)),
                "std": float(np.std(placeholder_counts)),
                "texts_with_placeholders": sum(1 for count in placeholder_counts if count > 0),
                "texts_with_placeholders_rate": sum(1 for count in placeholder_counts if count > 0) / len(generated_texts)
            }
        }

        return result

    def calculate_all_metrics(
        self,
        original_texts: List[str],
        generated_texts: List[str]
    ) -> Dict[str, Any]:
        """
        Calculate all utility metrics.

        Args:
            original_texts: List of original texts.
            generated_texts: List of generated texts.

        Returns:
            Dictionary with all metrics.
        """
        # Ensure the inputs have the same length
        if len(original_texts) != len(generated_texts):
            return {"error": "Number of original and generated texts must match"}

        # Calculate perplexity
        logger.info("Calculating perplexity...")
        perplexity_metrics = {}
        if self.use_perplexity:
            perplexity_metrics = self.calculate_perplexity(generated_texts)

        # Calculate semantic similarity
        logger.info("Calculating semantic similarity...")
        similarity_metrics = self.calculate_semantic_similarity(original_texts, generated_texts)

        # Calculate ROUGE scores
        logger.info("Calculating ROUGE scores...")
        rouge_metrics = {}
        if ROUGE_AVAILABLE:
            rouge_metrics = self.calculate_rouge_scores(original_texts, generated_texts)

        # Calculate n-gram diversity
        logger.info("Calculating n-gram diversity...")
        n_gram_metrics = self.calculate_n_gram_diversity(original_texts, generated_texts)

        # Calculate output statistics
        logger.info("Calculating output statistics...")
        output_stats = self.calculate_output_statistics(original_texts, generated_texts)

        # Combine all metrics
        all_metrics = {
            "perplexity": perplexity_metrics,
            "semantic_similarity": similarity_metrics,
            "rouge_scores": rouge_metrics,
            "n_gram_diversity": n_gram_metrics,
            "output_statistics": output_stats
        }

        return all_metrics

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class PrivacyFrameworkEvaluator:
    """
    Main evaluation pipeline for the synthetic data privacy framework.
    """

    def __init__(
        self,
        privacy_evaluator: Optional[PrivacyEvaluator] = None,
        utility_evaluator: Optional[UtilityEvaluator] = None,
        device: str = None,
        results_dir: str = "results"
    ):
        """
        Initialize the privacy framework evaluator.

        Args:
            privacy_evaluator: Preconfigured PrivacyEvaluator instance.
            utility_evaluator: Preconfigured UtilityEvaluator instance.
            device: Device to use for evaluations (cuda, cpu, mps).
            results_dir: Directory to store evaluation results.
        """
        # Initialize evaluators if not provided
        self.privacy_evaluator = privacy_evaluator or PrivacyEvaluator(device=device)
        self.utility_evaluator = utility_evaluator or UtilityEvaluator(device=device)

        # Setup results directory
        self.results_dir = results_dir
        os.makedirs(results_dir, exist_ok=True)

        # Store experimental configurations
        self.configurations = {}
        self.results = {}

    def add_configuration(
        self,
        config_id: str,
        config_name: str,
        config_description: str,
        configuration: Dict[str, Any]
    ):
        """
        Add a configuration to be evaluated.

        Args:
            config_id: Unique identifier for the configuration.
            config_name: Short name for the configuration.
            config_description: Description of the configuration.
            configuration: Configuration parameters.
        """
        self.configurations[config_id] = {
            "id": config_id,
            "name": config_name,
            "description": config_description,
            "configuration": configuration
        }

        logger.info(f"Added configuration: {config_name} (ID: {config_id})")

    def evaluate_configuration(
        self,
        config_id: str,
        original_data: pd.DataFrame,
        generated_data: Union[pd.DataFrame, List[str]],
        text_column: str,
        pii_metadata_column: str = "pii_metadata",
        canary_metadata_column: str = "canary_metadata",
        adversarial_results: Optional[Dict[str, Any]] = None
    ) -> Dict[str, Any]:
        """
        Evaluate a specific configuration.

        Args:
            config_id: Identifier of the configuration to evaluate.
            original_data: DataFrame with original data and metadata.
            generated_data: DataFrame or list with generated texts.
            text_column: Column containing the text.
            pii_metadata_column: Column containing PII metadata.
            canary_metadata_column: Column containing canary metadata.
            adversarial_results: Optional results from adversarial testing.

        Returns:
            Dictionary with evaluation results.
        """
        if config_id not in self.configurations:
            logger.error(f"Configuration ID {config_id} not found")
            return {"error": f"Configuration ID {config_id} not found"}

        config = self.configurations[config_id]
        logger.info(f"Evaluating configuration: {config['name']} (ID: {config_id})")

        # Extract original texts
        original_texts = original_data['body'].tolist()

        # Extract generated texts
        if isinstance(generated_data, pd.DataFrame):
            if text_column not in generated_data.columns:
                logger.error(f"Text column '{text_column}' not found in generated data")
                return {"error": f"Text column '{text_column}' not found in generated data"}

            generated_texts = generated_data[text_column].tolist()
        else:
            generated_texts = generated_data

        # Ensure we have matching counts
        if len(original_texts) != len(generated_texts):
            logger.error(f"Number of original texts ({len(original_texts)}) does not match number of generated texts ({len(generated_texts)})")
            return {"error": "Number of original and generated texts must match"}

        # Evaluate privacy metrics
        logger.info("Evaluating privacy metrics...")

        pii_leakage = self.privacy_evaluator.evaluate_pii_leakage(
            original_data=original_data,
            generated_texts=generated_texts,
            text_column=text_column,
            pii_metadata_column=pii_metadata_column
        )

        canary_leakage = self.privacy_evaluator.evaluate_canary_leakage(
            original_data=original_data,
            generated_texts=generated_texts,
            text_column=text_column,
            canary_metadata_column=canary_metadata_column
        )

        similarity_distribution = self.privacy_evaluator.evaluate_similarity_distribution(
            original_data=original_data,
            generated_texts=generated_texts,
            text_column=text_column
        )

        # Evaluate utility metrics
        logger.info("Evaluating utility metrics...")

        utility_metrics = self.utility_evaluator.calculate_all_metrics(
            original_texts=original_texts,
            generated_texts=generated_texts
        )

        # Combine all results
        results = {
            "config_id": config_id,
            "config_name": config["name"],
            "config_description": config["description"],
            "configuration": config["configuration"],
            "privacy_metrics": {
                "pii_leakage": pii_leakage,
                "canary_leakage": canary_leakage,
                "similarity_distribution": similarity_distribution,
                "adversarial_results": adversarial_results or {}
            },
            "utility_metrics": utility_metrics
        }

        # Store results
        self.results[config_id] = results

        # Save results to file
        self._save_results(config_id, results)

        return results

    def _save_results(self, config_id: str, results: Dict[str, Any]):
        """Save evaluation results to file."""
        results_path = os.path.join(self.results_dir, f"{config_id}_results.json")

        with open(results_path, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2)

        logger.info(f"Saved evaluation results to {results_path}")

    def create_comparison_charts(
        self,
        config_ids: List[str] = None,
        output_dir: Optional[str] = None
    ):
        """
        Create comparison charts for multiple configurations.

        Args:
            config_ids: List of configuration IDs to compare. If None, use all.
            output_dir: Directory to save charts. If None, use results_dir.
        """
        # Use all configurations if none specified
        if config_ids is None:
            config_ids = list(self.results.keys())

        # Skip if no results
        if not config_ids or not self.results:
            logger.warning("No results available for comparison")
            return

        # Get results for selected configurations
        configs_to_compare = []
        for config_id in config_ids:
            if config_id in self.results:
                configs_to_compare.append(self.results[config_id])
            else:
                logger.warning(f"Configuration {config_id} not found in results")

        if not configs_to_compare:
            logger.warning("No valid configurations found for comparison")
            return

        # Set output directory
        output_dir = output_dir or self.results_dir
        os.makedirs(output_dir, exist_ok=True)

        # Set plot style
        plt.style.use('seaborn-v0_8-darkgrid')

        # 1. Privacy-Utility Trade-off Chart
        self._create_privacy_utility_tradeoff(configs_to_compare, output_dir)

        # 2. PII Leakage Rate Comparison
        self._create_pii_leakage_comparison(configs_to_compare, output_dir)

        # 3. Semantic Similarity Distribution
        self._create_similarity_distribution_chart(configs_to_compare, output_dir)

        # 4. Perplexity Comparison
        self._create_perplexity_comparison(configs_to_compare, output_dir)

        # 5. Output Length Comparison
        self._create_output_length_comparison(configs_to_compare, output_dir)

    def _create_privacy_utility_tradeoff(
        self,
        configs: List[Dict[str, Any]],
        output_dir: str
    ):
        """Create privacy-utility trade-off chart."""
        # Extract data for the chart
        labels = []
        pii_leakage_rates = []
        semantic_similarities = []
        perplexities = []

        for config in configs:
            config_name = config["config_name"]
            labels.append(config_name)

            # Get PII leakage rate
            pii_leakage = config["privacy_metrics"]["pii_leakage"].get("pii_leakage_rate", 0)
            pii_leakage_rates.append(pii_leakage)

            # Get semantic similarity
            semantic_similarity = config["utility_metrics"]["semantic_similarity"].get("mean_similarity", 0)
            semantic_similarities.append(semantic_similarity)

            # Get perplexity (lower is better)
            perplexity = config["utility_metrics"]["perplexity"].get("mean_perplexity", 0)
            if perplexity > 0:
                # Normalize perplexity to 0-1 range (inverted)
                perplexities.append(1.0 / perplexity)
            else:
                perplexities.append(0)

        # Create the trade-off chart
        fig, ax = plt.subplots(figsize=(10, 6))

        scatter = ax.scatter(
            pii_leakage_rates,
            semantic_similarities,
            s=[p * 200 for p in perplexities],  # Size based on perplexity
            alpha=0.7
        )

        # Add labels for each point
        for i, label in enumerate(labels):
            ax.annotate(
                label,
                (pii_leakage_rates[i], semantic_similarities[i]),
                xytext=(5, 5),
                textcoords='offset points'
            )

        ax.set_xlabel('PII Leakage Rate (lower is better)')
        ax.set_ylabel('Semantic Similarity (higher is better)')
        ax.set_title('Privacy-Utility Trade-off')

        # Set axis ranges
        ax.set_xlim(-0.05, max(pii_leakage_rates) + 0.1)
        ax.set_ylim(min(semantic_similarities) - 0.1, 1.05)

        # Add grid
        ax.grid(True, linestyle='--', alpha=0.7)

        # Save the chart
        plt.tight_layout()
        output_path = os.path.join(output_dir, 'privacy_utility_tradeoff.png')
        plt.savefig(output_path, dpi=300)
        plt.close()

        logger.info(f"Created privacy-utility trade-off chart: {output_path}")

    def _create_pii_leakage_comparison(
        self,
        configs: List[Dict[str, Any]],
        output_dir: str
    ):
        """Create PII leakage comparison chart."""
        # Extract data for the chart
        labels = []
        pii_leakage_rates = []
        canary_leakage_rates = []

        for config in configs:
            config_name = config["config_name"]
            labels.append(config_name)

            # Get PII leakage rate
            pii_leakage = config["privacy_metrics"]["pii_leakage"].get("pii_leakage_rate", 0)
            pii_leakage_rates.append(pii_leakage)

            # Get canary leakage rate
            canary_leakage = config["privacy_metrics"]["canary_leakage"].get("total_leakage_rate", 0)
            canary_leakage_rates.append(canary_leakage)

        # Create the comparison chart
        fig, ax = plt.subplots(figsize=(12, 6))

        x = np.arange(len(labels))
        width = 0.35

        ax.bar(x - width/2, pii_leakage_rates, width, label='PII Leakage Rate')
        ax.bar(x + width/2, canary_leakage_rates, width, label='Canary Leakage Rate')

        ax.set_xlabel('Configuration')
        ax.set_ylabel('Leakage Rate')
        ax.set_title('PII and Canary Leakage Rates by Configuration')
        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=45, ha='right')
        ax.legend()

        # Set y-axis range
        ax.set_ylim(0, max(max(pii_leakage_rates), max(canary_leakage_rates)) + 0.1)

        # Add grid
        ax.grid(True, axis='y', linestyle='--', alpha=0.7)

        # Save the chart
        plt.tight_layout()
        output_path = os.path.join(output_dir, 'pii_leakage_comparison.png')
        plt.savefig(output_path, dpi=300)
        plt.close()

        logger.info(f"Created PII leakage comparison chart: {output_path}")

    def _create_similarity_distribution_chart(
        self,
        configs: List[Dict[str, Any]],
        output_dir: str
    ):
        """Create similarity distribution chart."""
        # Extract data for the chart
        all_similarities = {}

        for config in configs:
            config_name = config["config_name"]

            # Get similarity distribution
            similarities = config["privacy_metrics"]["similarity_distribution"].get("all_similarities", [])
            if similarities:
                all_similarities[config_name] = similarities

        if not all_similarities:
            logger.warning("No similarity data available for distribution chart")
            return

        # Create the distribution chart
        fig, ax = plt.subplots(figsize=(12, 6))

        for config_name, similarities in all_similarities.items():
            sns.kdeplot(similarities, label=config_name, ax=ax)

        ax.set_xlabel('Cosine Similarity')
        ax.set_ylabel('Density')
        ax.set_title('Similarity Distribution by Configuration')
        ax.legend()

        # Set x-axis range
        ax.set_xlim(0, 1)

        # Add grid
        ax.grid(True, linestyle='--', alpha=0.7)

        # Save the chart
        plt.tight_layout()
        output_path = os.path.join(output_dir, 'similarity_distribution.png')
        plt.savefig(output_path, dpi=300)
        plt.close()

        logger.info(f"Created similarity distribution chart: {output_path}")

    def _create_perplexity_comparison(
        self,
        configs: List[Dict[str, Any]],
        output_dir: str
    ):
        """Create perplexity comparison chart."""
        # Extract data for the chart
        labels = []
        perplexities = []

        for config in configs:
            config_name = config["config_name"]

            # Get perplexity
            perplexity = config["utility_metrics"]["perplexity"].get("mean_perplexity")
            if perplexity is not None:
                labels.append(config_name)
                perplexities.append(perplexity)

        if not perplexities:
            logger.warning("No perplexity data available for comparison chart")
            return

        # Create the comparison chart
        fig, ax = plt.subplots(figsize=(12, 6))

        ax.bar(labels, perplexities, alpha=0.7)

        ax.set_xlabel('Configuration')
        ax.set_ylabel('Perplexity (lower is better)')
        ax.set_title('Perplexity by Configuration')
        plt.xticks(rotation=45, ha='right')

        # Add grid
        ax.grid(True, axis='y', linestyle='--', alpha=0.7)

        # Save the chart
        plt.tight_layout()
        output_path = os.path.join(output_dir, 'perplexity_comparison.png')
        plt.savefig(output_path, dpi=300)
        plt.close()

        logger.info(f"Created perplexity comparison chart: {output_path}")

    def _create_output_length_comparison(
        self,
        configs: List[Dict[str, Any]],
        output_dir: str
    ):
        """Create output length comparison chart."""
        # Extract data for the chart
        labels = []
        orig_lengths = []
        gen_lengths = []
        placeholder_rates = []

        for config in configs:
            config_name = config["config_name"]
            labels.append(config_name)

            # Get output statistics
            output_stats = config["utility_metrics"]["output_statistics"]

            # Get word counts
            orig_len = output_stats.get("word_count", {}).get("original_mean", 0)
            gen_len = output_stats.get("word_count", {}).get("generated_mean", 0)
            orig_lengths.append(orig_len)
            gen_lengths.append(gen_len)

            # Get placeholder rate
            placeholder_rate = output_stats.get("placeholder_count", {}).get("texts_with_placeholders_rate", 0)
            placeholder_rates.append(placeholder_rate)

        # Create the comparison chart
        fig, ax1 = plt.subplots(figsize=(12, 6))

        x = np.arange(len(labels))
        width = 0.35

        ax1.bar(x - width/2, orig_lengths, width, label='Original Length', color='tab:blue', alpha=0.7)
        ax1.bar(x + width/2, gen_lengths, width, label='Generated Length', color='tab:orange', alpha=0.7)

        ax1.set_xlabel('Configuration')
        ax1.set_ylabel('Average Word Count')
        ax1.set_title('Text Length and Placeholder Rate by Configuration')
        ax1.set_xticks(x)
        ax1.set_xticklabels(labels, rotation=45, ha='right')
        ax1.legend(loc='upper left')

        # Create second y-axis for placeholder rate
        ax2 = ax1.twinx()
        ax2.plot(x, placeholder_rates, 'r-o', label='Placeholder Rate')
        ax2.set_ylabel('Placeholder Rate')
        ax2.set_ylim(0, max(placeholder_rates) + 0.1)
        ax2.legend(loc='upper right')

        # Add grid
        ax1.grid(True, axis='y', linestyle='--', alpha=0.7)

        # Save the chart
        plt.tight_layout()
        output_path = os.path.join(output_dir, 'output_length_comparison.png')
        plt.savefig(output_path, dpi=300)
        plt.close()

        logger.info(f"Created output length comparison chart: {output_path}")

    def create_summary_report(
        self,
        output_path: Optional[str] = None,
        config_ids: List[str] = None
    ):
        """
        Create a summary report of evaluation results.

        Args:
            output_path: Path to save the report. If None, use default.
            config_ids: List of configuration IDs to include. If None, use all.
        """
        # Use all configurations if none specified
        if config_ids is None:
            config_ids = list(self.results.keys())

        # Skip if no results
        if not config_ids or not self.results:
            logger.warning("No results available for report")
            return

        # Get results for selected configurations
        configs_to_report = []
        for config_id in config_ids:
            if config_id in self.results:
                configs_to_report.append(self.results[config_id])
            else:
                logger.warning(f"Configuration {config_id} not found in results")

        if not configs_to_report:
            logger.warning("No valid configurations found for report")
            return

        # Set output path
        if output_path is None:
            output_path = os.path.join(self.results_dir, 'evaluation_summary.json')

        # Create summary data
        summary = {
            "overview": {
                "num_configurations": len(configs_to_report),
                "configurations": [cfg["config_name"] for cfg in configs_to_report]
            },
            "privacy_metrics": {},
            "utility_metrics": {},
            "configurations": {}
        }

        # Add configuration details
        for config in configs_to_report:
            config_id = config["config_id"]
            config_name = config["config_name"]

            # Extract key metrics
            privacy_metrics = config["privacy_metrics"]
            utility_metrics = config["utility_metrics"]

            pii_leakage_rate = privacy_metrics["pii_leakage"].get("pii_leakage_rate", 0)
            canary_leakage_rate = privacy_metrics["canary_leakage"].get("total_leakage_rate", 0)
            similarity_mean = utility_metrics["semantic_similarity"].get("mean_similarity", 0)
            perplexity = utility_metrics["perplexity"].get("mean_perplexity", 0)

            # Add to summary
            summary["configurations"][config_id] = {
                "name": config_name,
                "description": config["config_description"],
                "key_metrics": {
                    "pii_leakage_rate": pii_leakage_rate,
                    "canary_leakage_rate": canary_leakage_rate,
                    "semantic_similarity": similarity_mean,
                    "perplexity": perplexity
                }
            }

        # Add comparative privacy metrics
        summary["privacy_metrics"] = {
            "pii_leakage_rates": {cfg["config_name"]: cfg["privacy_metrics"]["pii_leakage"].get("pii_leakage_rate", 0) for cfg in configs_to_report},
            "canary_leakage_rates": {cfg["config_name"]: cfg["privacy_metrics"]["canary_leakage"].get("total_leakage_rate", 0) for cfg in configs_to_report},
            "best_privacy_configuration": min([(cfg["config_name"], cfg["privacy_metrics"]["pii_leakage"].get("pii_leakage_rate", 0)) for cfg in configs_to_report], key=lambda x: x[1])[0]
        }

        # Add comparative utility metrics
        summary["utility_metrics"] = {
            "semantic_similarities": {cfg["config_name"]: cfg["utility_metrics"]["semantic_similarity"].get("mean_similarity", 0) for cfg in configs_to_report},
            "perplexities": {cfg["config_name"]: cfg["utility_metrics"]["perplexity"].get("mean_perplexity", 0) for cfg in configs_to_report if "mean_perplexity" in cfg["utility_metrics"]["perplexity"]},
            "best_utility_configuration": max([(cfg["config_name"], cfg["utility_metrics"]["semantic_similarity"].get("mean_similarity", 0)) for cfg in configs_to_report], key=lambda x: x[1])[0]
        }

        # Calculate privacy-utility balance
        privacy_utility_scores = []
        for cfg in configs_to_report:
            # Privacy score (0-1, higher is better)
            privacy_score = 1.0 - cfg["privacy_metrics"]["pii_leakage"].get("pii_leakage_rate", 0)

            # Utility score (0-1, higher is better)
            utility_score = cfg["utility_metrics"]["semantic_similarity"].get("mean_similarity", 0)

            # Combined score (simple average)
            combined_score = (privacy_score + utility_score) / 2

            privacy_utility_scores.append((cfg["config_name"], combined_score))

        # Find best balanced configuration
        best_balanced = max(privacy_utility_scores, key=lambda x: x[1])
        summary["best_balanced_configuration"] = {
            "name": best_balanced[0],
            "score": best_balanced[1]
        }

        # Save summary report
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2)

        logger.info(f"Saved evaluation summary report to {output_path}")

        return summary

In [ ]:
"""
Script to run the evaluation pipeline on different filter configurations.
"""


from pathlib import Path
from typing import List, Dict, Any

import pandas as pd

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('evaluation.log')
    ]
)
logger = logging.getLogger(__name__)





def create_configurations():
    """Create filter configurations for evaluation."""
    configs = {
        "baseline": {
            "name": "Baseline (No Filters)",
            "description": "No privacy filters applied",
            "pre_filter": None,
            "in_generation": {
                "temperature": 0.8,
                "top_k": 50,
                "top_p": 0.95,
                "repetition_penalty": 1.0,
                "blocked_words": []
            },
            "post_filter": None
        },
        "pre_only": {
            "name": "Pre-Generation Filter Only",
            "description": "Only pre-generation filtering applied",
            "pre_filter": {
                "use_token_level": True,
                "type_specific_placeholders": True
            },
            "in_generation": {
                "temperature": 0.8,
                "top_k": 50,
                "top_p": 0.95,
                "repetition_penalty": 1.0,
                "blocked_words": []
            },
            "post_filter": None
        },
        "post_only": {
            "name": "Post-Generation Filter Only",
            "description": "Only post-generation filtering applied",
            "pre_filter": None,
            "in_generation": {
                "temperature": 0.8,
                "top_k": 50,
                "top_p": 0.95,
                "repetition_penalty": 1.0,
                "blocked_words": []
            },
            "post_filter": {
                "base_threshold": 0.92,
                "reapply_pii_detection": True,
                "use_token_level_redaction": True
            }
        },
        "pre_post": {
            "name": "Pre+Post Filters",
            "description": "Pre-generation and post-generation filters applied",
            "pre_filter": {
                "use_token_level": True,
                "type_specific_placeholders": True
            },
            "in_generation": {
                "temperature": 0.8,
                "top_k": 50,
                "top_p": 0.95,
                "repetition_penalty": 1.0,
                "blocked_words": []
            },
            "post_filter": {
                "base_threshold": 0.92,
                "reapply_pii_detection": True,
                "use_token_level_redaction": True
            }
        },
        "all_filters": {
            "name": "All Filters",
            "description": "All privacy filters applied (pre, in, post)",
            "pre_filter": {
                "use_token_level": True,
                "type_specific_placeholders": True
            },
            "in_generation": {
                "temperature": 0.8,
                "top_k": 50,
                "top_p": 0.95,
                "repetition_penalty": 1.2,
                "blocked_words": ["password", "social security", "credit card", "address", "phone number", "email"]
            },
            "post_filter": {
                "base_threshold": 0.92,
                "reapply_pii_detection": True,
                "use_token_level_redaction": True
            }
        },
        "all_strict": {
            "name": "All Filters (Strict)",
            "description": "All privacy filters with stricter settings",
            "pre_filter": {
                "use_token_level": True,
                "type_specific_placeholders": True
            },
            "in_generation": {
                "temperature": 0.7,
                "top_k": 40,
                "top_p": 0.9,
                "repetition_penalty": 1.3,
                "blocked_words": ["password", "social security", "credit card", "address", "phone number", "email"]
            },
            "post_filter": {
                "base_threshold": 0.85,  # Lower threshold = more filtering
                "reapply_pii_detection": True,
                "use_token_level_redaction": True
            }
        },
        "adaptive": {
            "name": "Adaptive Thresholds",
            "description": "All filters with content-dependent adaptive thresholds",
            "pre_filter": {
                "use_token_level": True,
                "type_specific_placeholders": True
            },
            "in_generation": {
                "temperature": 0.8,
                "top_k": 50,
                "top_p": 0.95,
                "repetition_penalty": 1.2,
                "blocked_words": ["password", "social security", "credit card", "address", "phone number", "email"]
            },
            "post_filter": {
                "base_threshold": 0.92,
                "adaptive_thresholds": {
                    "technical": 0.95,
                    "general": 0.9,
                    "personal": 0.85
                },
                "reapply_pii_detection": True,
                "use_token_level_redaction": True
            }
        }
    }

    return configs







In [ ]:
os.makedirs('results', exist_ok=True)


    # Inject sensitive data
processed_data = pd.read_json("processed_dataset.json")

    # Get configurations
all_configs = create_configurations()
configs_to_run=None
if configs_to_run:
        configs = {k: v for k, v in all_configs.items() if k in configs_to_run}
        if not configs:
            logger.error(f"None of the specified configurations found: {configs_to_run}")
            logger.info(f"Available configurations: {list(all_configs.keys())}")
            print("not working")
else:
        configs = all_configs

In [ ]:
logger.info("Setting up evaluators...")
privacy_evaluator = PrivacyEvaluator()
utility_evaluator = UtilityEvaluator()
evaluator = PrivacyFrameworkEvaluator(
        privacy_evaluator=privacy_evaluator,
        utility_evaluator=utility_evaluator,
        results_dir='results'
  )

    # Add configurations to evaluator
for config_id, config in configs.items():
    evaluator.add_configuration(
        config_id=config_id,
        config_name=config["name"],
        config_description=config["description"],
        configuration=config
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
for config_id, config in configs.items():
        # Generate text with this configuration
    logger.info(f"Processing configuration: {config['name']} (ID: {config_id})")

    generation_results = pd.read_json(f"{config_id}_generated.json")
        # Evaluate configuration
    logger.info(f"Evaluating configuration: {config['name']} (ID: {config_id})")
    evaluation_results = evaluator.evaluate_configuration(
            config_id=config_id,
            original_data=processed_data,
            generated_data=generation_results,
            text_column="generated_text",
            pii_metadata_column="pii_metadata",
            canary_metadata_column="canary_metadata"
    )

    # Create comparison charts
logger.info("Creating comparison charts...")
evaluator.create_comparison_charts()

    # Create summary report
logger.info("Creating summary report...")
summary = evaluator.create_summary_report()

logger.info(f"Evaluation complete. Results saved to {output_dir}")



Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

NameError: name 'output_dir' is not defined

In [ ]:
    # Print summary of best configurations
print("\n----- Evaluation Summary -----")
print(f"Best privacy configuration: {summary['privacy_metrics']['best_privacy_configuration']}")
print(f"Best utility configuration: {summary['utility_metrics']['best_utility_configuration']}")
print(f"Best balanced configuration: {summary['best_balanced_configuration']['name']} (Score: {summary['best_balanced_configuration']['score']:.4f})")


----- Evaluation Summary -----
Best privacy configuration: Adaptive Thresholds
Best utility configuration: Baseline (No Filters)
Best balanced configuration: All Filters (Score: 0.7701)


In [ ]:
print(summary)

{'overview': {'num_configurations': 7, 'configurations': ['Baseline (No Filters)', 'Pre-Generation Filter Only', 'Post-Generation Filter Only', 'Pre+Post Filters', 'All Filters', 'All Filters (Strict)', 'Adaptive Thresholds']}, 'privacy_metrics': {'pii_leakage_rates': {'Baseline (No Filters)': 0.0, 'Pre-Generation Filter Only': 0.0, 'Post-Generation Filter Only': 0.0, 'Pre+Post Filters': 0.0, 'All Filters': 0.0, 'All Filters (Strict)': 0.0, 'Adaptive Thresholds': 0.0}, 'canary_leakage_rates': {'Baseline (No Filters)': 0.0, 'Pre-Generation Filter Only': 0.0, 'Post-Generation Filter Only': 0.0, 'Pre+Post Filters': 0.0, 'All Filters': 0.0, 'All Filters (Strict)': 0.0, 'Adaptive Thresholds': 0.0}, 'best_privacy_configuration': 'Baseline (No Filters)'}, 'utility_metrics': {'semantic_similarities': {'Baseline (No Filters)': 0.6876073502935469, 'Pre-Generation Filter Only': 0.6224781853705644, 'Post-Generation Filter Only': 0.5849954656604678, 'Pre+Post Filters': 0.6001203941460699, 'All Filt